In [ ]:
# 1. Setup Environment
import sys
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Clone repository if not already present
if not os.path.exists('/content/sam-3d-body'):
    !git clone https://github.com/facebookresearch/sam-3d-body.git /content/sam-3d-body

# Install dependencies (uncomment if needed)
# !pip install -r /content/sam-3d-body/requirements.txt
# !pip install -e /content/sam-3d-body

# Add repo to python path
sys.path.append('/content/sam-3d-body')
os.chdir('/content/sam-3d-body')

In [ ]:
# 2. Imports
import torch
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sam_3d_body import load_sam_3d_body, SAM3DBodyEstimator
from tools.vis_utils import visualize_sample_together
from tools.build_detector import HumanDetector

In [ ]:
# 3. Configuration
# Define paths to your model and assets in Google Drive
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/PoseEstimation/SAM-Body3d/model.ckpt"
mhr_path = "/content/drive/MyDrive/Colab Notebooks/PoseEstimation/SAM-Body3d/assets/mhr_model.pt"

# Define input and output video paths
# Replace with your actual video path in Drive or upload one
video_path = "/content/drive/MyDrive/Colab Notebooks/PoseEstimation/SAM-Body3d/input_video.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/PoseEstimation/SAM-Body3d/output_video.mp4"

# Optional: Path to model_config.yaml if it's not automatically found near the checkpoint
# model_config_path = "/content/drive/MyDrive/Colab Notebooks/PoseEstimation/SAM-Body3d/model_config.yaml"

In [ ]:
# 4. Load Model
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load the SAM 3D Body model
# We pass the paths directly. If you have a specific config path, pass model_config_path=model_config_path
model, model_cfg = load_sam_3d_body(
    checkpoint_path, 
    device=device, 
    mhr_path=mhr_path
)

# Initialize Human Detector (ViTDet)
print("Loading Human Detector...")
human_detector = HumanDetector(name="vitdet", device=device)

# Create the Estimator
estimator = SAM3DBodyEstimator(
    sam_3d_body_model=model,
    model_cfg=model_cfg,
    human_detector=human_detector,
    # human_segmentor=None, # Add segmentor if needed
    # fov_estimator=None,   # Add FOV estimator if needed
)
print("Model loaded successfully!")

In [ ]:
# 5. Process Video
def process_video_colab(estimator, video_path, output_path):
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Processing video: {video_path}")
    print(f"Output will be saved to: {output_path}")
    
    # Use the optimized generator (process_video) which avoids clearing cache every frame
    generator = estimator.process_video(
        video_path,
        bbox_thr=0.5,
        use_mask=False,
        verbose=False 
    )
    
    for frame, outputs in tqdm(generator, total=total_frames):
        # Visualize results on the frame
        rend_img = visualize_sample_together(frame, outputs, estimator.faces)
        writer.write(rend_img.astype(np.uint8))
        
    cap.release()
    writer.release()
    print("Done.")

# Run the processing
process_video_colab(estimator, video_path, output_path)

In [ ]:
# 6. (Optional) Download or View Result
# If running in Colab, you can download the file to your local machine
try:
    from google.colab import files
    files.download(output_path)
except ImportError:
    print("Not running in Colab or files module not available.")